# Stedin Distribution Transformer
In this document we will implement a magnetostatic model for the distribution transformer geometry of the Gmsh exercises. This model is heavily inspired by the COMSOL model implemented in the [thesis](http://resolver.tudelft.nl/uuid:15b25b42-e04b-4ff2-a187-773bc170f061) of Max van Dijk.
Because the end goal is quite complex, we will build this model in several steps:
1. Linear magnetostatics
2. Linear magnetostatics and eddy currents
3. Non-linear magnetostatics and eddy currents
4. Linear magnetostatics with voltage-driven coils
5. Non- magnetostatics with voltage-driven coils

The relevant theory is discussed in each of the sections. The geometry is redefined below to allow playing around with the mesh density.

In [ ]:
using gmsh

using LinearAlgebra, SparseArrays
using WriteVTK

using BenchmarkTools

# Geometry Definition
See exercise on Gmsh.

# Load Geometry
To provide efficient access to the geometry data, we pre-process it and store it in a `mesh_data` struct. This contains information about the nodes (number and position), as well as the elements (number, connectivity, and physical domain).

In [ ]:
gmsh.finalize()
gmsh.initialize()

In [ ]:
# mesh_data struct
#  saves the nodes and elements of a gmsh geometry, for easy passing around in functions
struct mesh_data
    nnodes  # number of nodes
    xnode   # array of x coordinates
    ynode   # array of y coordinates
    
    nelements   # number of elements
    #element_connectivity  # array of connectivity for each element
    e_group     # array containing the physical group number of each element
    elements    # <new addition> more conveniently structured connectivity array
end

# Loads nodes, elements, and element physical groups from gmsh and stores them in a mesh_data struct
function get_mesh_data()
    #..2/11 Get and sort the mesh nodes
    #..Observe that although the mesh is two-dimensional,
    #..the z-coordinate that is equal to zero is stored as well.
    #..Observe that the coordinates are stored contiguously for computational
    #..efficiency
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)
    #..sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
    sorted = sortslices(tosort , dims = 1);
    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]
    
    #..4/12 Get the mesh elements
    #..observe that we get all the two-dimensional triangular elements from the mesh
    element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
    nelements = length(element_ids[1])

    #..5/12 Create groups of elements for the subdomains
    #..for loop that creates a vector describing which physical group an element belongs to
    ngroup1 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 1)
    ngroup2 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 2)
    ngroup3 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 3)
    ngroup4 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 4)
    ngroup5 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 5)
    ngroup6 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 6)
    ngroup7 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 7)
    ngroup8 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 8)
    ngroup9 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 9)
    ngroup10 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 10)
    ngroup11 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 11)
    ngroup12 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 12)
    ngroup13 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 13)
    ngroup14 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 14)
    ngroup15 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 15)
    ngroup16 = gmsh.model.mesh.getNodesForPhysicalGroup(2, 16)
    e_group = zeros(1,nelements)
    
    elements = [zeros(Int, 3) for i in 1:nelements];
    
    for element_id in 1:nelements
        node1_id = element_connectivity[1][3*(element_id-1)+1]
        node2_id = element_connectivity[1][3*(element_id-1)+2]
        node3_id = element_connectivity[1][3*(element_id-1)+3]
        
        # Store connectivity in a convenient format
        elements[element_id] = [node1_id, node2_id, node3_id];
        
        # Determine which physical group the element belongs to
        G1  = sum(node1_id.== ngroup1[1])+sum(node2_id.== ngroup1[1])+sum(node3_id.== ngroup1[1]) # Oil
        G2  = sum(node1_id.== ngroup2[1])+sum(node2_id.== ngroup2[1])+sum(node3_id.== ngroup2[1]) # Core
        G3  = sum(node1_id.== ngroup3[1])+sum(node2_id.== ngroup3[1])+sum(node3_id.== ngroup3[1]) # HV winding phase 1 left
        G4  = sum(node1_id.== ngroup4[1])+sum(node2_id.== ngroup4[1])+sum(node3_id.== ngroup4[1]) # HV winding phase 1 right
        G5  = sum(node1_id.== ngroup5[1])+sum(node2_id.== ngroup5[1])+sum(node3_id.== ngroup5[1]) # HV winding phase 2 left
        G6  = sum(node1_id.== ngroup6[1])+sum(node2_id.== ngroup6[1])+sum(node3_id.== ngroup6[1]) # HV winding phase 2 right
        G7  = sum(node1_id.== ngroup7[1])+sum(node2_id.== ngroup7[1])+sum(node3_id.== ngroup7[1]) # HV winding phase 3 left
        G8  = sum(node1_id.== ngroup8[1])+sum(node2_id.== ngroup8[1])+sum(node3_id.== ngroup8[1]) # HV winding phase 3 right
        G9  = sum(node1_id.== ngroup9[1])+sum(node2_id.== ngroup9[1])+sum(node3_id.== ngroup9[1]) # LV winding phase 1 left
        G10 = sum(node1_id.== ngroup10[1])+sum(node2_id.== ngroup10[1])+sum(node3_id.== ngroup10[1]) # LV winding phase 1 right
        G11 = sum(node1_id.== ngroup11[1])+sum(node2_id.== ngroup11[1])+sum(node3_id.== ngroup11[1]) # LV winding phase 2 left
        G12 = sum(node1_id.== ngroup12[1])+sum(node2_id.== ngroup12[1])+sum(node3_id.== ngroup12[1]) # LV winding phase 2 right
        G13 = sum(node1_id.== ngroup13[1])+sum(node2_id.== ngroup13[1])+sum(node3_id.== ngroup13[1]) # LV winding phase 3 left
        G14 = sum(node1_id.== ngroup14[1])+sum(node2_id.== ngroup14[1])+sum(node3_id.== ngroup14[1]) # LV winding phase 3 right
        
        if G1 == 3
            e_group[element_id] = 1;
        elseif G2 == 3
            e_group[element_id] = 2;
        elseif G3 == 3
            e_group[element_id] = 3;
        elseif G4 == 3
            e_group[element_id] = 4;
        elseif G5 == 3
            e_group[element_id] = 5;
        elseif G6 == 3
            e_group[element_id] = 6;
        elseif G7 == 3
            e_group[element_id] = 7;
        elseif G8 == 3
            e_group[element_id] = 8;
        elseif G9 == 3
            e_group[element_id] = 9;
        elseif G10 == 3
            e_group[element_id] = 10;
        elseif G11 == 3
            e_group[element_id] = 11;
        elseif G12 == 3
            e_group[element_id] = 12;
        elseif G13 == 3
            e_group[element_id] = 13;
        elseif G14 == 3
            e_group[element_id] = 14;
        end
    end
    
    return mesh_data(nnodes, xnode, ynode, nelements, e_group, elements)
end

# Linear 2D FEM (Time-Harmonic Magnetostatics)
Just like in the one-dimensional time-harmonic case, we assume that the fields are varying sinusoidally in time at a fixed frequency $\omega$. Taking into account both the imposed source current density $\mathbf{J}_0$ and the conduction current density $\mathbf{J}_c = \sigma \mathbf{E}$, we obtain the following differential equation.
$$ \nabla \times \left[ \frac{1}{\mu} \nabla \times \mathbf{A} \right] = \mathbf{J}_0 + \mathbf{J}_c $$
Under the assumption that the flow of current is oriented along the $z$ axis, and that the geometry is in the $xy$ plane, this can be rewritten as
$$ -\nabla \cdot \left[ \frac{1}{\mu} \nabla A_z \right] = J_0 + J_c $$

## Conduction Current
From Maxwell's equations, we find that the electric field can be written as
$$ \mathbf{E} = -\nabla V - \frac{\partial \mathbf{A}}{\partial t} = -j\omega A_z$$
where we assume that the electrostatic field $-\nabla V$ is negligible. Substituting this into the differential equation presented above,
$$ -\nabla \cdot \left[ \nu \nabla A_z \right] + j\omega\sigma A_z = J_0  $$

## Variational Form
$$ -\int_\Omega \nabla \cdot \left[ \nu \nabla A_z \right] \phi_k d\Omega + j\omega \int_\Omega \sigma A_z \phi_k d\Omega = \int_\Omega J_0 \phi_k d\Omega \qquad 1 \le k \le N $$
Rewriting and re-ordering the equation according to the integration by parts discussed during the lecture, we obtain the variational form consisting of a bilinear term, a linear term, and a boundary contribution.
$$ \int_\Omega \nu \nabla A_z \cdot \nabla\phi_k d\Omega + j\omega \int_\Omega \sigma A_z \phi_k d\Omega = \int_\Omega J_0 \phi_k d\Omega + \oint_\Gamma \nu \left[ \nabla A_z \cdot \mathbf{n} \right] d\Gamma \qquad 1 \le k \le N $$
In this exercise we will impose a homogeneous Dirichlet condition the boundary of the geometry. Therefore, the boundary contribution term will be zero.

## Matrix Assembly
The integral terms above correspond to the stiffness matrix $A$, mass matrix $M$, and source vector $f$, respectively. Calculation of the element contributions and assembly into the global matrix are discussed in a separate notebook.

In [ ]:
function fem2d(mshdata, sourceperelement, reluctivityperelement, conductivityperelement, omega)
    A = spzeros(Complex{Float64}, mshdata.nnodes, mshdata.nnodes)
    f = zeros(Complex{Float64}, mshdata.nnodes, 1)

    xnode = mshdata.xnode;
    ynode = mshdata.ynode;
    
    # Perform a loop over the elements
    for (element_id, nodes) in enumerate(mshdata.elements)
        # Retrieve global numbering of the local nodes of the current element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];

        # Retrieve the x and y coordinates of the local nodes of the current element
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        # Compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        # Compute local vector contribution floc of the current element
        floc = area_id/3*sourceperelement[element_id] * [1; 1; 1]

        # Compute local matrix contribution Aloc of the current element
        Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
        Emat[3,:] .= 0;
        Bloc = area_id*reluctivityperelement[element_id]*(transpose(Emat)*Emat);
        Cloc = 1im * area_id / 3 * conductivityperelement[element_id] * omega * Diagonal(ones(3));

        # Add local contribution to f and A
        f[nodes]       += floc;
        A[nodes,nodes] += (Bloc + Cloc);
    end

    # Handle the boundary conditions
    bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = Diagonal(ones(size(bnd_node_ids)))
    f[bnd_node_ids] .= 0;
    
    # Compute the numerical solution
    u = A \ f
    
    return u;
end

## Post-processing
After calculating the vector potential $A_z$, we are interested in knowing the other fields. Most importantly, the magnetic flux density $\mathbf{B}$ and corresponding field intensity $\mathbf{H}$. 

### Magnetic Flux Density
The flux density is defined in Maxwell's equations as
$$ \mathbf{B} = \nabla \times \mathbf{A} = \frac{\partial A_z}{\partial y} \hat{\mathbf{x}} - \frac{\partial A_z}{\partial x} \hat{\mathbf{y}} $$
Since $A_z = u^h = \sum_{i = 1}^N u_i \phi_i(x,y)$, where $i$ are the nodes, we can write the two components of the vector as
$$ B_x = \sum_{i = 1}^N u_i \frac{\partial \phi_i(x,y)}{\partial y} \qquad B_x = -\sum_{i = 1}^N u_i \frac{\partial \phi_i(x,y)}{\partial x} $$
On each element, we see three (linear) shape functions $\phi_i(x,y) = a_i x + b_i y + c_i$ with $i = 1,2,3$ (note: local numbering). Then the vector components are defined on the element $e_k$ as follows:
$$ B_x(x,y) = \left[ u_1 b_1 + u_2 b_2 + u_3 b_3 \right] \qquad B_y(x,y) = -\left[ u_1 a_1 + u_2 a_2 + u_3 a_3 \right] \qquad (x,y) \in e_k $$

### Current Density
Finally, it is also interesting to observe the current density in each of the elements, which consists of two components: imposed source current density and conduction current density. On the element $k$,
$$ J_k = J_{0,k} + J_{c,k} = J_{0,k} + j\omega\sigma \cdot \frac{1}{3} \left[ A_{k,1} + A_{k,2} + A_{k,3} \right] $$
where the conduction current density is approximated by averaging the vector potential over the element.

In [ ]:
function process(mshdata, u, sourceperelement, reluctivityperelement, conductivityperelement, omega)
    Bx = zeros(Complex{Float64}, mshdata.nelements);
    By = zeros(Complex{Float64}, mshdata.nelements);
    
    Jel = zeros(mshdata.nelements);
    
    # Perform a loop over the elements
    for (element_id, nodes) in enumerate(mshdata.elements)
        # Retrieve global numbering of the local nodes of the current element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];
        
        # Get x and y coordinates of the three nodes
        xnode = mshdata.xnode; ynode = mshdata.ynode;
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        # Compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        # Calculate shape function parameters
        Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
    
        # Calculate Bx and By from the solution coefficients and the shape function parameters
        c = u[[node1_id, node2_id, node3_id]];
        Bx[element_id] = sum(c .* Emat[2,:]);
        By[element_id] = -sum(c .* Emat[1,:]);
        
        # Calculate eddy current loss
        sigma = conductivityperelement[element_id];
        Jel[element_id] = norm(sourceperelement[element_id] + omega * sigma * 1/3 * sum(c));
    end
    
    # H is related to B through the reluctivity
    Hx = reluctivityperelement' .* Bx;
    Hy = reluctivityperelement' .* By;
    
    # Energy is 0.5 * dot(B, H)
    Wm = 0.5 * (Bx .* Hx .+ By .* Hy);
    
    return (Bx,By), (Hx, Hy), Wm, Jel;
end

# Linear FEM without Eddy Currents
Initially, we set the conductivity $\sigma$ to be zero everywhere. In this case we have magnetostatics without eddy currents.

In [ ]:
gmsh.open("geo/transformer_stedin.msh")
mshdata = get_mesh_data();

In [ ]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

# Source current density J
#  One term for each of the windings, with a positive and negative part
#  Note the phase shift between the phases
sourcefunction(group_id) = Jp * exp(1im * 2pi/3) * (-1 * (group_id==3) + 1 * (group_id==4)) + 
                           Jp * (-1 * (group_id==5) + 1 * (group_id==6)) + 
                           Jp * exp(-1im * 2pi/3) * (-1 * (group_id==7) + 1 * (group_id==8)) + 
                           Js * exp(1im * 2pi/3) * (1 * (group_id==9) - 1 * (group_id==10)) +
                           Js * (1 * (group_id==11) - 1 * (group_id==12)) + 
                           Js * exp(-1im * 2pi/3) * (1 * (group_id==13) - 1 * (group_id==14));
sourceperelement = map(sourcefunction, mshdata.e_group);

# Relative permeability model
mu0 = 4e-7 * pi;
mur = 1000;       # Relative permeability of the core
reluctivityfunction(group_id) = (1 / mu0) + (1/(mu0*mur) - 1/mu0) * (group_id == 2)
reluctivityperelement = map(reluctivityfunction, mshdata.e_group);

# Conductivity
conductivityfunction(group_id) = 0;
conductivityperelement = map(conductivityfunction, mshdata.e_group);

In [ ]:
# Calculate the vector potential
u = fem2d(mshdata, sourceperelement, reluctivityperelement, conductivityperelement, omega);

# Post-process for magnetic field and current density
B, H, Wm, Jel = process(mshdata, u, sourceperelement, reluctivityperelement, conductivityperelement, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));

In [ ]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("images/transformer/transformer1.vtu", points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Jel", VTKCellData()]   = Jel;

# Save the file
outfiles = vtk_save(vtkfile);

![Magnetic Field: Linear, No Eddy Currents](images/transformer/transformer1.png)

# Linear FEM with Eddy Currents
Next, we will model the magnetic core with a certain conductivity $\sigma_{core}$. For grain-oriented silicon steel, the conductivity is typically $2 \cdot 10^6\ \mathrm{S/m}$. However, to minimize eddy current loss, the core is laminated in the $xy$ plane. This significantly reduces the conductivity in the $z$ direction, which we model as a reduced conductivity $\sigma_{core} = 0.1\ \mathrm{S/m}$.

In [ ]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

# Source current density J
#  One term for each of the windings, with a positive and negative part
#  Note the phase shift between the phases
sourcefunction(group_id) = Jp * exp(1im * 2pi/3) * (-1 * (group_id==3) + 1 * (group_id==4)) + 
                           Jp * (-1 * (group_id==5) + 1 * (group_id==6)) + 
                           Jp * exp(-1im * 2pi/3) * (-1 * (group_id==7) + 1 * (group_id==8)) + 
                           Js * exp(1im * 2pi/3) * (1 * (group_id==9) - 1 * (group_id==10)) +
                           Js * (1 * (group_id==11) - 1 * (group_id==12)) + 
                           Js * exp(-1im * 2pi/3) * (1 * (group_id==13) - 1 * (group_id==14));
sourceperelement = map(sourcefunction, mshdata.e_group);

# Relative permeability model
mu0 = 4e-7 * pi;
mur = 1000;       # Relative permeability of the core
reluctivityfunction(group_id) = (1 / mu0) + (1/(mu0*mur) - 1/mu0) * (group_id == 2)
reluctivityperelement = map(reluctivityfunction, mshdata.e_group);

# Conductivity
conductivityfunction(group_id) = 0.1;
conductivityperelement = map(conductivityfunction, mshdata.e_group);

In [ ]:
# Calculate the vector potential
u = fem2d(mshdata, sourceperelement, reluctivityperelement, conductivityperelement, omega);

# Post-process for magnetic field and current density
B, H, Wm, Jel = process(mshdata, u, sourceperelement, reluctivityperelement, conductivityperelement, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));

In [ ]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("images/transformer/transformer2.vtu", points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Jel", VTKCellData()]   = Jel;

# Save the file
outfiles = vtk_save(vtkfile);

![Magnetic Field: Linear, with Eddy Currents](images/transformer/transformer2.png)

# Non-Linear FEM
The next step is the inclusion of a non-linear $BH$ characteristic which models the saturation of the magnetic core material. In doing so, the reluctivity $\nu(x,y)$ becomes a function of the magnetic flux density $\nu(x,y,|B|)$. As a result, the $A$ matrix will also be a function of $|B|$ (or equivalently, of $A_z$).
To find a solution to this non-linear system of equations we will apply iterative substitution, just like in the 1D case. This is preferred over Newtons method, because time-harmonic problems are not sufficiently smooth to easily calculate a Jacobian by automatic differentiation.

To obtain good performance, the matrix assembly and solving is split into two parts:
1. Pre-assemble all contributions which are _constant_. Most parts of the system of equations are constant during the non-linear solving and can be pre-allocated and assembled, instead of recalculating them every on every linear step.
2. During each linear step, calculate only those contributions that are dependent on $|B|$ and assemble the global system accordingly.

Note that we implement a damped substitution method to prevent oscillatory behaviour. This is necessary because the time-harmonic problem is not very smooth. Choosing a proper damping factor $\alpha$ is critical to achieve convergence in the least number of steps.
$$ u_k = \alpha u_k + (1 - \alpha) u_{k-1} $$

In [ ]:
gmsh.open("geo/transformer_stedin.msh")
mshdata = get_mesh_data();

In [ ]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

# Source current density J
#  One term for each of the windings, with a positive and negative part
#  Note the phase shift between the phases
sourcefunction(group_id) = Jp * exp(1im * 2pi/3) * (-1 * (group_id==3) + 1 * (group_id==4)) + 
                           Jp * (-1 * (group_id==5) + 1 * (group_id==6)) + 
                           Jp * exp(-1im * 2pi/3) * (-1 * (group_id==7) + 1 * (group_id==8)) + 
                           Js * exp(1im * 2pi/3) * (1 * (group_id==9) - 1 * (group_id==10)) +
                           Js * (1 * (group_id==11) - 1 * (group_id==12)) + 
                           Js * exp(-1im * 2pi/3) * (1 * (group_id==13) - 1 * (group_id==14));
sourceperelement = map(sourcefunction, mshdata.e_group);

# Permeability function
bh_a = 2.12e-4; 
bh_b = 7.358;
bh_c = 1.18e7;
mu0  = 4e-7 * pi;
fmur_core(B) = 1 / (bh_a + (1 - bh_a) * B^(2*bh_b) / (B^(2*bh_b) + bh_c));
fmu(group_id, B) = mu0 + (fmur_core(B) - 1) * mu0 * (group_id == 2);
reluctivityfunction(group_id, B) = 1 / fmu(group_id, B);

# Conductivity
conductivityfunction(group_id) = 0.1;
conductivityperelement = map(conductivityfunction, mshdata.e_group);

In [ ]:
# Pre-calculate the matrix contributions that are constant in the non-linear problem
function precalc_contribs(mshdata, sourceperelement, conductivityperelement, omega)
    elids = 1:mshdata.nelements;
    
    Bmat = [zeros(9) for i in elids];
    Cmat = [zeros(Complex{Float64}, 9) for i in elids];
    Emat = [zeros(3,3) for i in elids];
    
    f = zeros(Complex{Float64}, mshdata.nnodes)
    
    xnode = mshdata.xnode;
    ynode = mshdata.ynode;
    
    #..8/12 Perform a loop over the elements
    for (element_id, nodes) in enumerate(mshdata.elements)
        #....retrieve global numbering of the local nodes of the current element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];

        #....retrieve the x and y coordinates of the local nodes of the current element
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        #....compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        #....compute local vector contribution floc of the current element
        floc = area_id/3*sourceperelement[element_id]*[1; 1; 1]

        #....compute local matrix contribution Aloc of the current element
        Eloc = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
        Eloc[3,:] .= 0;
        Bloc = area_id*(transpose(Eloc)*Eloc);
        Cloc = 1im * area_id / 3 * conductivityperelement[element_id] * omega * Diagonal(ones(3));

        #....add local contribution to f and A
        f[nodes] += floc;
        
        Bmat[element_id] = Bloc[:];
        Cmat[element_id] = Cloc[:];
        Emat[element_id] = Eloc;
    end
    
    return Bmat, Cmat, Emat, f;
end

In [ ]:
function assemble_matrices(u, I, J, Bmat, Cmat, Emat, N)
    # Calculate |B| and the corresponding nu(B) on each element
    c = map(el -> u[el], mshdata.elements)
    Bx = map((c, E) -> norm(sum(c .* E[2, :])), c, Emat)
    By = map((c, E) -> -norm(sum(c .* E[1, :])), c, Emat)
    Bnorm = map((Bx, By) -> norm(sqrt(Bx^2 + By^2)), Bx, By)
    reluctivityperelement = map(reluctivityfunction, mshdata.e_group, Bnorm);
    
    # Generate matrix contributions
    Vb = reduce(vcat, map((B, nu) -> B * nu, Bmat, reluctivityperelement));
    Vc = reduce(vcat, Cmat);
    V  = Vb + Vc;
    
    return sparse(I, J, V, N, N);
end

function apply_boundaries(A, bnd_node_ids, e)
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = e;
    
    return A;
end

In [ ]:
Bmat, Cmat, Emat, f = precalc_contribs(mshdata, sourceperelement, conductivityperelement, omega);

# Generate index vectors
I_ = reduce(vcat, view.(mshdata.elements, Ref([1, 2, 3, 1, 2, 3, 1, 2, 3])))
J_ = reduce(vcat, view.(mshdata.elements, Ref([1, 1, 1, 2, 2, 2, 3, 3, 3])))
N = mshdata.nnodes;

# Generate vectors for boundary condition 
bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);
e = Diagonal(ones(size(bnd_node_ids)));

In [ ]:
function nonlinsolve(mshdata, I, J, Bmat, Cmat, Emat, N, bnd_node_ids, e, alpha)
    A = assemble_matrices(zeros(mshdata.nnodes), I, J, Bmat, Cmat, Emat, N);
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = e;
    f[bnd_node_ids] .= 0;

    u = A \ f;

    du = 1e6;
    Niter = 1;

    tol = 1e-6;
    Nmax = 200;
    
    uhist = zeros(N);
    
    while (du > tol) && (Niter < Nmax)
        uprev = u;
        uhist = uhist * alpha + u * (1 - alpha);    # Provide some damping to prevent oscillation between two solutions

        A = assemble_matrices(uhist, I, J, Bmat, Cmat, Emat, N);
        A = apply_boundaries(A, bnd_node_ids, e);
        
        u = A \ f;

        du = norm(u - uprev);
        Niter += 1;
        print("#$Niter: $du\n")
    end
    
    return u;
end

In [ ]:
u = nonlinsolve(mshdata, I_, J_, Bmat, Cmat, Emat, N, bnd_node_ids, e, 0.9);

In [ ]:
# Calculate the final nu(B) for each element for post-processing
c = map(el -> u[el], mshdata.elements)
Bx = map((c, E) -> norm(sum(c .* E[2, :])), c, Emat)
By = map((c, E) -> -norm(sum(c .* E[1, :])), c, Emat)
Bnorm = map((Bx, By) -> norm(sqrt(Bx^2 + By^2)), Bx, By)
reluctivityperelement = map(reluctivityfunction, mshdata.e_group, Bnorm);

In [ ]:
B, H, Wm, Jel = process(mshdata, u, sourceperelement, reluctivityperelement', conductivityperelement, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));

In [ ]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("images/transformer/transformer3.vtu", points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Jel", VTKCellData()]   = Jel;
vtkfile["mu_r", VTKCellData()]  = 1 ./ (mu0 * reluctivityperelement);

# Save the file
outfiles = vtk_save(vtkfile);

![Magnetic Field: Non-linear, with Eddy Currents](images/transformer/transformer3.png)

# Voltage-Driven Coils
In reality, the windings of the transformer are driven by voltage sources ($10.75\ \mathrm{kV}$ primary, and $420\ \mathrm{V}$ secondary). Until now we have assumed a certain amount of current was flowing through each of the windings, but in practice this depends on the loading of the transformer.

Therefore, we want to determine the coil current dynamically, based on the voltage $V_{src}$ that is applied to the winding and the voltage $V_{ind}$ induced in the winding by the changing magnetic field. Assuming a winding resistance $R$, the coil current becomes
$$ I_{coil} = \frac{V_{src} - V_{ind}}{R} $$
In this equation, $I_{coil}$ is the unknown we would like to solve, whereas $V_{src}$ and $R$ are known. The induced voltage depends on the magnetic vector potential, which is also unknown:
$$ V_{ind} = \frac{N \ell}{A_{coil}} \int_{A_{coil}} (\mathbf{E} \cdot \mathbf{e}_{coil}) d\Omega $$
where $\mathbf{e}_{coil}$ is the unit vector in the direction of the coil winding. In the current planar geometry, this can be rewritten as
$$ V_{ind} = \frac{N \ell}{A_{coil}} \int_{A_{coil}} E_z d\Omega = \frac{j\omega N \ell}{A_{coil}} \int_{A_{coil}} A_z d\Omega $$

## Modifying the Linear System
For each coil in the geometry, we must add an equation/row in the matrix (for the induced voltage) and a unknown (the coil current). To couple this coil current to the geometry, an extra column is also required. If the original system has size $N$, then adding $k$ coils results in an $(N+k) \times (N+k)$ matrix and vectors of size $N+k$. 

The problem we want to implement has three secondary coils which are supplied with a voltage, and three open-circuited primary coils (i.e. imposed $I = 0$). Therefore, we need three extra coil current/voltage equations.
$$ \tilde{A} \tilde{u} = \tilde{f} \implies \begin{bmatrix}
     A            & \mathbf{a}_1 & \mathbf{a}_2 & \mathbf{a}_3 \\
     \mathbf{b}_1 & 1            & 0            & 0            \\
     \mathbf{b}_2 & 0            & 1            & 0            \\
     \mathbf{b}_3 & 0            & 0            & 1            \\
\end{bmatrix} \begin{bmatrix}
    \mathbf{u} \\ I_{1} \\ I_{2} \\ I_{3}
\end{bmatrix} = \begin{bmatrix}
    \mathbf{f} \\ V_{1} \\ V_{2} \\ V_{3}
\end{bmatrix} $$

## Implementation
The additional required columns $\mathbf{a}_i$ and rows $\mathbf{b}_i$ can be derived from the equations presented above. Every element in the coil $i$ contributes to the entries corresponding to its nodes, just like the other matrix contributions. For element $j$ belonging to coil $i$, the contribution to $\mathbf{a}_i$ is
$$ J_z = \frac{N_i I_{i}}{A_i} \implies a_{i,j} = \frac{N_i I_i}{A_i} \frac{\operatorname{area}(e_j)}{3} \begin{bmatrix}
    1 \\ 1 \\ 1 \\
\end{bmatrix} $$
And the contribution to $\mathbf{b}_i$ follows from the discretization of the induced voltage integral
$$ b_{i,j} = -\frac{j\omega N \ell}{R_i A_i} \frac{\operatorname{area}(e_j)}{3} \begin{bmatrix}
    1 & 1 & 1
\end{bmatrix} $$

In [ ]:
function fem2d_voltage(mshdata, sourceperelement, reluctivityperelement, conductivityperelement, omega)
    Nnodes = mshdata.nnodes;
    Ncoils = 3;
    
    N = Nnodes + Ncoils;
    A = spzeros(Complex{Float64}, N, N)
    f = zeros(Complex{Float64}, N, 1)

    xnode = mshdata.xnode;
    ynode = mshdata.ynode;
    
    CFF  = 0.3;
    Vsec = 420 * sqrt(2 / 3);
    Rsec = 1.2999e-3 / CFF;
    Asec = Awlv;
    lsec = 0.4;
    
    for (element_id, nodes) in enumerate(mshdata.elements)
        #....retrieve global numbering of the local nodes of the current element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];
        
        #....retrieve the x and y coordinates of the local nodes of the current element
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        #....compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        #....compute local vector contribution floc of the current element
        floc = area_id/3*sourceperelement[element_id]*[1; 1; 1]

        #....compute local matrix contribution Aloc of the current element
        Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
        Emat[3,:] .= 0;
        Bloc = area_id*reluctivityperelement[element_id]*(transpose(Emat)*Emat);
        Cloc = 1im * area_id / 3 * conductivityperelement[element_id] * omega * Diagonal(ones(3));

        #....add local contribution to f and A
        f[nodes]        += floc;
        A[nodes, nodes] += (Bloc + Cloc);
        
        # LV winding 1 left (positive)
        if(mshdata.e_group[element_id] == 9)
            A[nodes, Nnodes + 1] += Ns / Asec * area_id/3 * [1;1;1];
            A[Nnodes + 1, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 1 right (negative)
        elseif(mshdata.e_group[element_id] == 10)
            A[nodes, Nnodes + 1] += -Ns / Asec * area_id/3 * [1;1;1];
            A[Nnodes + 1, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 2 right (positive)
        elseif(mshdata.e_group[element_id] == 11)
            A[nodes, Nnodes + 2] += Ns / Asec * area_id/3 * [1;1;1];
            A[Nnodes + 2, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 2 right (negative)
        elseif(mshdata.e_group[element_id] == 12)
            A[nodes, Nnodes + 2] += -Ns / Asec * area_id/3 * [1;1;1];
            A[Nnodes + 2, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 3 right (positive)
        elseif(mshdata.e_group[element_id] == 13)
            A[nodes, Nnodes + 3] += Ns / Asec * area_id/3 * [1;1;1];
            A[Nnodes + 3, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 3 right (negative)
        elseif(mshdata.e_group[element_id] == 14)
            A[nodes, Nnodes + 3] += -Ns / Asec * area_id/3 * [1;1;1];
            A[Nnodes + 3, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
        end
    end
    
    f[Nnodes + 1] = Vsec * exp(1im * 2pi/3) / Rsec;
    f[Nnodes + 2] = Vsec / Rsec;
    f[Nnodes + 3] = Vsec * exp(1im * -2pi/3) / Rsec;
    A[Nnodes+1, Nnodes+1] = 1;
    A[Nnodes+2, Nnodes+2] = 1;
    A[Nnodes+3, Nnodes+3] = 1;

    #..9/12 Handle the boundary conditions
    bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = Diagonal(ones(size(bnd_node_ids)))
    f[bnd_node_ids] .= 0;
    
    #..10/12 Compute the numerical solution
    u = A \ f
    
    return u, A;
end

In [ ]:
function process_voltage(mshdata, u, sourceperelement, reluctivityperelement, conductivityperelement, omega)
    Nnodes = mshdata.nnodes;
    
    Bx = zeros(Complex{Float64}, mshdata.nelements);
    By = zeros(Complex{Float64}, mshdata.nelements);
    
    Jel = zeros(Complex{Float64}, mshdata.nelements);
    
    for (element_id, nodes) in enumerate(mshdata.elements)
        # Get nodes constituting the element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];
        
        # Get x and y coordinates of the three nodes
        xnode = mshdata.xnode; ynode = mshdata.ynode;
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        # Compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        # Calculate shape function parameters
        Emat = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
    
        # Calculate Bx and By from the solution coefficients and the shape function parameters
        c = u[[node1_id, node2_id, node3_id]];
        Bx[element_id] = sum(c .* Emat[2,:]);
        By[element_id] = -sum(c .* Emat[1,:]);
        
        # Calculate eddy current loss
        sigma = conductivityperelement[element_id];
        Jel[element_id] = sourceperelement[element_id] + 1im * omega * sigma * 1/3 * sum(c);
        
        # LV winding 1 left (positive)
        if(mshdata.e_group[element_id] == 9)
            Jel[element_id] += u[Nnodes + 1] * Ns / Awlv;
            
        # LV winding 1 right (negative)
        elseif(mshdata.e_group[element_id] == 10)
            Jel[element_id] += -u[Nnodes + 1] * Ns / Awlv;
            
        # LV winding 2 right (positive)
        elseif(mshdata.e_group[element_id] == 11)
            Jel[element_id] += u[Nnodes + 2] * Ns / Awlv;
            
        # LV winding 2 right (negative)
        elseif(mshdata.e_group[element_id] == 12)
            Jel[element_id] += -u[Nnodes + 2] * Ns / Awlv;
            
        # LV winding 3 right (positive)
        elseif(mshdata.e_group[element_id] == 13)
            Jel[element_id] += u[Nnodes + 3] * Ns / Awlv;
            
        # LV winding 3 right (negative)
        elseif(mshdata.e_group[element_id] == 14)
            Jel[element_id] += -u[Nnodes + 3] * Ns / Awlv;
        end
    end
    
    Jel = norm.(Jel);
    
    # H is related to B through the reluctivity
    Hx = reluctivityperelement' .* Bx;
    Hy = reluctivityperelement' .* By;
    
    # Energy is 0.5 * dot(B, H)
    Wm = 0.5 * (Bx .* Hx .+ By .* Hy);
    
    return (Bx,By), (Hx, Hy), Wm, Jel;
end

## Simulate

In [ ]:
gmsh.open("geo/transformer_stedin.msh")
mshdata = get_mesh_data();

In [ ]:
Ip = 0*17.54;       # Primary peak phase current
Is = 777.62;      # Secondary peak phase current

Np = 266;
Ns = 6;
omega = 2*pi*50;

Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

sourcefunction(group_id) = 0;
sourceperelement = map(sourcefunction, mshdata.e_group);

mu0 = 4e-7 * pi;
mur = 1000;       # Relative permeability of the core
reluctivityfunction(group_id) = (1 / mu0) + (1/(mu0*mur) - 1/mu0) * (group_id == 2)
reluctivityperelement = map(reluctivityfunction, mshdata.e_group);

sigma_core = 0.1;
conductivityfunction(group_id) = sigma_core * (group_id == 2);
conductivityperelement = map(conductivityfunction, mshdata.e_group);

In [ ]:
u, A = fem2d_voltage(mshdata, sourceperelement, reluctivityperelement, conductivityperelement, omega);
B, H, Wm, Jel = process_voltage(mshdata, u, sourceperelement, reluctivityperelement, conductivityperelement, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));
Hnorm = norm.(sqrt.(H[1].^2 + H[2].^2));

Isec1 = u[mshdata.nnodes + 1];
Isec2 = u[mshdata.nnodes + 2];
Isec3 = u[mshdata.nnodes + 3];
u = u[1:mshdata.nnodes];

In [ ]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("images/transformer/transformer4.vtu", points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Hnorm", VTKCellData()] = Hnorm;
vtkfile["Jel", VTKCellData()]   = Jel;

# Save the file
outfiles = vtk_save(vtkfile);

In [ ]:
print("I_{sec1}: ", norm(Isec1), " A, ", angle(Isec1) / pi * 180, " deg\n")
print("I_{sec2}: ", norm(Isec2), " A, ", angle(Isec2) / pi * 180, " deg\n")
print("I_{sec3}: ", norm(Isec3), " A, ", angle(Isec3) / pi * 180, " deg\n")

In [ ]:
ph12 = (angle(Isec1) - angle(Isec2)) / pi * 180;
ph13 = (angle(Isec1) - angle(Isec3)) / pi * 180;
ph23 = (angle(Isec2) - angle(Isec3)) / pi * 180;

print("phase differences: $ph12, $ph13, $ph23\n")

This is not entirely as expected: The amplitudes of the phase currents should be equal, with a phase difference of $120^\circ$ between the phases.

![Magnetic Field: Voltage-driven Coils](images/transformer/transformer4.png)

# Non-linear FEM + Voltage driven coils
Finally, we combine the non-linear FEM and the voltage-driven coils discussed in the two sections above.

In [ ]:
# Pre-calculate the matrix contributions that are constant in the non-linear problem
function precalc_contribs(mshdata, sourceperelement, conductivityperelement, omega, nc)
    elids = 1:mshdata.nelements;
    
    Bmat = [zeros(9) for i in elids];
    Cmat = [zeros(Complex{Float64}, 9) for i in elids];
    Emat = [zeros(3,3) for i in elids];
    
    
    CFF  = 0.3;
    Vsec = 420 * sqrt(2 / 3);
    Rsec = 1.2999e-3 / CFF;
    Asec = Awlv;
    lsec = 0.4;
    
    ceqs_r = zeros(Complex{Float64}, nc, mshdata.nnodes);
    ceqs_c = zeros(Complex{Float64}, mshdata.nnodes, nc);
    
    f = zeros(Complex{Float64}, mshdata.nnodes + nc)
    
    xnode = mshdata.xnode;
    ynode = mshdata.ynode;
    
    #..8/12 Perform a loop over the elements
    for (element_id, nodes) in enumerate(mshdata.elements)
        #....retrieve global numbering of the local nodes of the current element
        node1_id = nodes[1]; node2_id = nodes[2]; node3_id = nodes[3];

        #....retrieve the x and y coordinates of the local nodes of the current element
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id];

        #....compute surface area of the current element
        x12 = xnode2 - xnode1; x13 = xnode3-xnode1;
        y12 = ynode2 - ynode1; y13 = ynode3-ynode1;
        area_id = x12*y13 - x13*y12; area_id = abs(area_id)/2

        #....compute local vector contribution floc of the current element
        floc = area_id/3*sourceperelement[element_id]*[1; 1; 1]

        #....compute local matrix contribution Aloc of the current element
        Eloc = [[xnode1;xnode2;xnode3] [ynode1;ynode2;ynode3] [1;1;1]] \ UniformScaling(1.);
        Eloc[3,:] .= 0;
        Bloc = area_id*(transpose(Eloc)*Eloc);
        Cloc = 1im * area_id / 3 * conductivityperelement[element_id] * omega * Diagonal(ones(3));

        #....add local contribution to f and A
        f[nodes] += floc;
        
        # LV winding 1 left (positive)
        if(mshdata.e_group[element_id] == 9)
            ceqs_c[nodes, 1] += Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[1, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 1 right (negative)
        elseif(mshdata.e_group[element_id] == 10)
            ceqs_c[nodes, 1] += -Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[1, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 2 right (positive)
        elseif(mshdata.e_group[element_id] == 11)
            ceqs_c[nodes, 2] += Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[2, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 2 right (negative)
        elseif(mshdata.e_group[element_id] == 12)
            ceqs_c[nodes, 2] += -Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[2, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 3 right (positive)
        elseif(mshdata.e_group[element_id] == 13)
            ceqs_c[nodes, 3] += Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[3, nodes] += -1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
            
        # LV winding 3 right (negative)
        elseif(mshdata.e_group[element_id] == 14)
            ceqs_c[nodes, 3] += -Ns / Asec * area_id/3 * [1;1;1];
            ceqs_r[3, nodes] += 1im * omega * Ns * lsec / (Asec * Rsec) * area_id/3 * [1;1;1];
        end
        
        Bmat[element_id] = Bloc[:];
        Cmat[element_id] = Cloc[:];
        Emat[element_id] = Eloc;
    end
    
    f[mshdata.nnodes + 1] = Vsec * exp(1im * 2pi/3) / Rsec;
    f[mshdata.nnodes + 2] = Vsec / Rsec;
    f[mshdata.nnodes + 3] = Vsec * exp(1im * -2pi/3) / Rsec;
    
    return Bmat, Cmat, Emat, f, ceqs_r, ceqs_c;
end

In [ ]:
function assemble_matrices(u, I, J, Bmat, Cmat, Emat, N, nc)
    c = map(el -> u[el], mshdata.elements)
    Bx = map((c, E) -> norm(sum(c .* E[2, :])), c, Emat)
    By = map((c, E) -> -norm(sum(c .* E[1, :])), c, Emat)
    Bnorm = map((Bx, By) -> norm(sqrt(Bx^2 + By^2)), Bx, By)
    reluctivityperelement = map(reluctivityfunction, mshdata.e_group, Bnorm);
    
    # Generate matrix contributions
    Vb = reduce(vcat, map((B, nu) -> B * nu, Bmat, reluctivityperelement));
    Vc = reduce(vcat, Cmat);
    V  = Vb + Vc;
    
    return sparse(I, J, V, N + nc, N + nc);
end

function apply_boundaries(A, bnd_node_ids, e)
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = e;
    #f[bnd_node_ids] .= 0;
    
    return A#, f;
end

function apply_circuits(A, N, nc, ceqs_r, ceqs_c)
    idx_i = N .+ (1:nc);
    idx_j = 1:N;
    
    A[idx_i, idx_j] = ceqs_r;
    A[idx_j, idx_i] = ceqs_c;
    A[idx_i, idx_i] = Diagonal(ones(nc));
    
    return A;
end

In [ ]:
gmsh.open("geo/transformer_stedin.msh")
mshdata = get_mesh_data();

In [ ]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

# Source current density J
# Because the current is now determined by the circuit coupling, we set the imposed current density to zero.
sourcefunction(group_id) = 0
sourceperelement = map(sourcefunction, mshdata.e_group);

# Permeability function
bh_a = 1 / 47e3;
bh_b = 3.6;
bh_c = 2.1e8;
mu0  = 4e-7 * pi;
fmur_core(B) = 1 / (bh_a + (1 - bh_a) * B^(2*bh_b) / (B^(2*bh_b) + bh_c));

fmu(group_id, B) = mu0 + (fmur_core(B) - 1) * mu0 * (group_id == 2);
reluctivityfunction(group_id, B) = 1 / fmu(group_id, B);

sigma_core = 0;
conductivityfunction(group_id) = sigma_core * (group_id == 2);
conductivityperelement = map(conductivityfunction, mshdata.e_group);

In [ ]:
Bmat, Cmat, Emat, f, ceqs_r, ceqs_c = precalc_contribs(mshdata, sourceperelement, conductivityperelement, omega, 3);

# Generate index vectors
I_ = reduce(vcat, view.(mshdata.elements, Ref([1, 2, 3, 1, 2, 3, 1, 2, 3])))
J_ = reduce(vcat, view.(mshdata.elements, Ref([1, 1, 1, 2, 2, 2, 3, 3, 3])))
N = mshdata.nnodes;

# Generate vectors for boundary condition 
bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);
e = Diagonal(ones(size(bnd_node_ids)));

In [ ]:
function nonlinsolve(mshdata, I, J, Bmat, Cmat, Emat, ceqs_r, ceqs_c, N, bnd_node_ids, e, alpha)
    A = assemble_matrices(zeros(mshdata.nnodes), I, J, Bmat, Cmat, Emat, N, 3);
    A[bnd_node_ids,:] .= 0;
    A[bnd_node_ids,bnd_node_ids] = e;
    f[bnd_node_ids] .= 0;
    A = apply_circuits(A, N, 3, ceqs_r, ceqs_c);

    u = A \ f;

    du = 1e6;
    Niter = 1;

    tol = 1e-6;
    Nmax = 200;
    
    uhist = zeros(N + 3);
    
    while (du > tol) && (Niter < Nmax)
        uprev = u;
        uhist = uhist * alpha + u * (1 - alpha);    # Provide some damping to prevent oscillation between two solutions

        A = assemble_matrices(uhist, I, J, Bmat, Cmat, Emat, N, 3);
        A = apply_boundaries(A, bnd_node_ids, e);
        A = apply_circuits(A, N, 3, ceqs_r, ceqs_c);
        
        u = A \ f;

        du = norm(u - uprev);
        Niter += 1;
        print("#$Niter: $du\n")
    end
    
    return u;
end

In [ ]:
u = nonlinsolve(mshdata, I_, J_, Bmat, Cmat, Emat, ceqs_r, ceqs_c, N, bnd_node_ids, e, 0.9);

In [ ]:
c = map(el -> u[el], mshdata.elements)
Bx = map((c, E) -> norm(sum(c .* E[2, :])), c, Emat)
By = map((c, E) -> -norm(sum(c .* E[1, :])), c, Emat)
Bnorm = map((Bx, By) -> norm(sqrt(Bx^2 + By^2)), Bx, By)
reluctivityperelement = map(reluctivityfunction, mshdata.e_group, Bnorm);

B, H, Wm, Jel = process_voltage(mshdata, u, sourceperelement, reluctivityperelement', conductivityperelement, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));
Hnorm = norm.(sqrt.(H[1].^2 + H[2].^2));

Isec1 = u[mshdata.nnodes + 1];
Isec2 = u[mshdata.nnodes + 2];
Isec3 = u[mshdata.nnodes + 3];
u = u[1:mshdata.nnodes];

In [ ]:
print("I_{sec1}: ", norm(Isec1), " A, ", angle(Isec1) / pi * 180, " deg\n")
print("I_{sec2}: ", norm(Isec2), " A, ", angle(Isec2) / pi * 180, " deg\n")
print("I_{sec3}: ", norm(Isec3), " A, ", angle(Isec3) / pi * 180, " deg\n")

In [ ]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid("images/transformer/transformer5.vtu", points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Hnorm", VTKCellData()] = Hnorm;
vtkfile["Jel", VTKCellData()]   = Jel;
vtkfile["mu_r", VTKCellData()]  = 1 ./ (mu0 * reluctivityperelement);

# Save the file
outfiles = vtk_save(vtkfile);

Magnetic flux density
![Magnetic Field: Voltage-driven Coils + Non-linear](images/transformer/transformer5.png)

Current density
![Current density: Voltage-driven Coils + Non-linear](images/transformer/normJ_opencircuit.png)

Relative permeability
![Relative permeability: Voltage-driven Coils + Non-linear](images/transformer/mur_opencircuit.png)